In [ ]:
import pandas as pd
import numpy as np
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score
from sklearn.linear_model import LogisticRegression
from ipca_classes_update import IPCA_v1
import warnings
from xgboost import XGBClassifier
warnings.filterwarnings('ignore')

def prepare_earnings_surprise_data(filepath):
    """
    Load and prepare data for earnings surprise prediction using IPCA
    """
    # Load data
    data = pd.read_csv(filepath)
    print(f"Loaded data: {data.shape[0]} observations, {data.shape[1]} features")
    
    # Create date column and sort
    data['date'] = pd.to_datetime(data[['year', 'month']].assign(day=1))
    data = data.sort_values(['date', 'permno']).reset_index(drop=True)
    
    # Create earnings surprise binary target
    # 1 if actual EPS > median estimate, 0 otherwise
    data['earnings_surprise'] = np.where(
        (data['eps_actual'].notna()) & (data['eps_medest'].notna()),
        (data['eps_actual'] > data['eps_medest']).astype(int),
        np.nan
    )
    
    print(f"Earnings surprise distribution:")
    print(data['earnings_surprise'].value_counts(dropna=False))
    
    # Define characteristic variables (excluding forward-looking and identifiers)
    exclude_vars = [
        # Target/forward-looking variables
        'ret_eom', 'stock_exret', 'earnings_surprise',
        'eps_medest', 'eps_meanest', 'eps_stdevest', 'eps_actual',
        
        # Identifiers and date variables
        'permno', 'CUSIP', 'stock_ticker', 'comp_name', 
        'year', 'month', 'date', 'SHRCD', 'EXCHCD',
        
        # Market-wide variables
        'RF', 'size_port'
    ]
    
    # Get characteristic variables
    char_vars = [col for col in data.columns if col not in exclude_vars]
    print(f"Using {len(char_vars)} characteristic variables for IPCA")
    
    return data, char_vars

def run_ipca_earnings_prediction(data, char_vars, K=6, min_obs_per_date=50, 
                                oos_start_year=2010, oos_window=60):
    """
    Run IPCA analysis for earnings surprise prediction
    """
    # Filter data with valid earnings surprise and sufficient characteristics
    valid_data = data.dropna(subset=['earnings_surprise'] + char_vars[:20])  # Require at least 20 non-missing chars
    print(f"Data after filtering: {valid_data.shape[0]} observations")
    
    # Filter dates with sufficient cross-section
    date_counts = valid_data.groupby('date').size()
    valid_dates = date_counts[date_counts >= min_obs_per_date].index
    valid_data = valid_data[valid_data['date'].isin(valid_dates)]
    print(f"Using {len(valid_dates)} dates with sufficient cross-section")
    
    # Create multi-index dataset for IPCA
    ipca_data = valid_data.set_index(['date', 'permno'])[['earnings_surprise'] + char_vars]
    
    # Handle missing values by forward-filling within each stock
    ipca_data = ipca_data.groupby(level=1).fillna(method='ffill')
    
    # Rank transform characteristics to [-0.5, 0.5] by date
    char_data = ipca_data[char_vars].copy()
    for date in char_data.index.get_level_values(0).unique():
        date_mask = char_data.index.get_level_values(0) == date
        for var in char_vars:
            if char_data.loc[date_mask, var].notna().sum() > 10:  # Sufficient non-missing
                ranks = char_data.loc[date_mask, var].rank(method='dense') - 1
                max_rank = ranks.max()
                if max_rank > 0:
                    char_data.loc[date_mask, var] = (ranks / max_rank) - 0.5
                else:
                    char_data.loc[date_mask, var] = 0
    
    # Combine target with transformed characteristics
    ipca_input = pd.concat([ipca_data[['earnings_surprise']], char_data], axis=1)
    ipca_input = ipca_input.dropna()
    
    print(f"Final IPCA dataset: {ipca_input.shape[0]} observations")
    
    # Initialize IPCA
    ipca = IPCA_v1(ipca_input, return_column='earnings_surprise', add_constant=True)
    
    # Split data for OOS analysis
    oos_dates = ipca_input.index.get_level_values(0) >= pd.to_datetime(f'{oos_start_year}-01-01')
    
    if oos_dates.sum() > 0:
        print("Running out-of-sample IPCA estimation...")
        # Out-of-sample estimation
        results = ipca.fit(
            K=K, 
            OOS=True, 
            OOS_window='recursive', 
            OOS_window_specs=oos_window,
            R_fit=True,
            dispIters=True,
            dispItersInt=50
        )
    else:
        print("Running in-sample IPCA estimation...")
        # In-sample estimation
        results = ipca.fit(K=K, R_fit=True, dispIters=True)
    
    return results, ipca, ipca_input

def select_optimal_k(data, char_vars, k_range=range(4, 16), 
                    min_obs_per_date=100, oos_start_year=2010, oos_window=60):
    """
    Select optimal number of factors K using cross-validation
    """
    print("\n" + "="*50)
    print("SELECTING OPTIMAL NUMBER OF FACTORS (K)")
    print("="*50)
    
    k_results = {}
    
    for k in k_range:
        print(f"\nTesting K = {k}...")
        try:
            # Run IPCA with current K
            results, ipca, ipca_input_k = run_ipca_earnings_prediction(
                data, char_vars, K=k, min_obs_per_date=min_obs_per_date,
                oos_start_year=oos_start_year, oos_window=oos_window
            )
            
            # Quick evaluation
            prediction_results = evaluate_ipca_factors_for_prediction(
                results, ipca_input_k, ipca, K=k, verbose=False
            )
            
            if prediction_results and 'metrics' in prediction_results:
                auc = prediction_results['metrics']['auc']
                r2_managed = results['xfits']['R2_Total']
                r2_returns = results['rfits']['R2_Total']
                
                k_results[k] = {
                    'auc': auc,
                    'r2_managed': r2_managed,
                    'r2_returns': r2_returns,
                    'score': auc  # Primary metric for selection
                }
                
                print(f"  AUC: {auc:.3f}, R²(managed): {r2_managed:.3f}, R²(returns): {r2_returns:.3f}")
            else:
                k_results[k] = {'auc': 0, 'r2_managed': 0, 'r2_returns': 0, 'score': 0}
                print(f"  Failed to evaluate K={k}")
                
        except Exception as e:
            print(f"  Error with K={k}: {str(e)}")
            k_results[k] = {'auc': 0, 'r2_managed': 0, 'r2_returns': 0, 'score': 0}
    
    # Find optimal K
    if k_results:
        optimal_k = max(k_results.keys(), key=lambda x: k_results[x]['score'])
        
        print(f"\n" + "="*30)
        print("K SELECTION RESULTS:")
        print("="*30)
        for k in sorted(k_results.keys()):
            metrics = k_results[k]
            marker = " ← OPTIMAL" if k == optimal_k else ""
            print(f"K={k:2d}: AUC={metrics['auc']:.3f}, R²(mgd)={metrics['r2_managed']:.3f}, R²(ret)={metrics['r2_returns']:.3f}{marker}")
        
        print(f"\nSelected optimal K = {optimal_k}")
        return optimal_k, k_results
    else:
        print("No valid results found, defaulting to K=6")
        return 6, {}

def evaluate_ipca_factors_for_prediction(results, ipca_input, ipca, K=6, verbose=True):
    """
    Use IPCA factors to predict earnings surprise
    """
    if verbose:
        print("\n" + "="*50)
        print("EVALUATING IPCA FACTORS FOR EARNINGS PREDICTION")
        print("="*50)
    
    # Extract factor loadings and create factor scores
    gamma = results['Gamma']
    if isinstance(gamma.index, pd.MultiIndex):
        # OOS case - use latest Gamma
        latest_date = gamma.index.get_level_values(0).max()
        gamma_final = gamma.loc[latest_date]
    else:
        # In-sample case
        gamma_final = gamma
    
    print(f"Factor loadings shape: {gamma_final.shape}")
    if verbose:
        print(f"Top characteristics for each factor:")
        for i in range(min(K, gamma_final.shape[1])):
            factor_name = gamma_final.columns[i]
            top_chars = gamma_final.iloc[:, i].abs().nlargest(5)
            print(f"\nFactor {factor_name}:")
            for char, loading in top_chars.items():
                print(f"  {char}: {loading:.3f}")
    
    # Create factor scores for prediction
    char_vars = gamma_final.index.tolist()
    if 'Constant' in char_vars:
        char_vars.remove('Constant')
    
    # Calculate factor scores
    factor_scores_list = []
    
    for date in ipca_input.index.get_level_values(0).unique():
        date_data = ipca_input.loc[date]
        if date_data.shape[0] > 10:  # Sufficient observations
            # Get characteristics matrix
            X = date_data[char_vars].values
            # Add constant
            X_with_const = np.column_stack([X, np.ones(X.shape[0])])
            
            # Calculate factor scores
            factor_scores = X_with_const @ gamma_final.values
            
            # Create DataFrame
            factor_df = pd.DataFrame(
                factor_scores, 
                index=date_data.index,
                columns=gamma_final.columns
            )
            factor_df['date'] = date
            factor_df['earnings_surprise'] = date_data['earnings_surprise'].values
            factor_scores_list.append(factor_df)
    
    if len(factor_scores_list) == 0:
        if verbose:
            print("No valid data for factor score calculation")
        return None
    
    # Combine all factor scores
    all_factor_scores = pd.concat(factor_scores_list, ignore_index=True)
    all_factor_scores = all_factor_scores.dropna()
    
    if verbose:
        print(f"\nFactor scores dataset: {all_factor_scores.shape[0]} observations")
    
    # Split into train/test
    train_data = all_factor_scores[all_factor_scores['date'] < pd.to_datetime('2015-01-01')]
    test_data = all_factor_scores[all_factor_scores['date'] >= pd.to_datetime('2015-01-01')]
    
    if len(train_data) == 0 or len(test_data) == 0:
        if verbose:
            print("Insufficient data for train/test split")
        return all_factor_scores
    
    if verbose:
        print(f"Train set: {len(train_data)} observations")
        print(f"Test set: {len(test_data)} observations")
    
    # Prepare features (exclude date and target)
    factor_cols = gamma_final.columns.tolist()
    X_train = train_data[factor_cols]
    y_train = train_data['earnings_surprise']
    X_test = test_data[factor_cols]
    y_test = test_data['earnings_surprise']
    
    # Train logistic regression
# With this:
    xgb = XGBClassifier(random_state=42, eval_metric='logloss')
    xgb.fit(X_train, y_train)
    y_pred_proba = xgb.predict_proba(X_test)[:, 1]
    y_pred = xgb.predict(X_test)
    # Evaluate
    accuracy = accuracy_score(y_test, y_pred)
    precision = precision_score(y_test, y_pred)
    recall = recall_score(y_test, y_pred)
    f1 = f1_score(y_test, y_pred)
    auc = roc_auc_score(y_test, y_pred_proba)
    
    if verbose:
        print(f"\nPREDICTION RESULTS:")
        print(f"Accuracy: {accuracy:.3f}")
        print(f"Precision: {precision:.3f}")
        print(f"Recall: {recall:.3f}")
        print(f"F1-Score: {f1:.3f}")
        print(f"AUC-ROC: {auc:.3f}")
        
        # Feature importance
        print(f"\nFACTOR IMPORTANCE (Logistic Regression Coefficients):")
        for factor, coef in zip(factor_cols, lr.coef_[0]):
            print(f"{factor}: {coef:.3f}")
    
    return {
        'factor_scores': all_factor_scores,
        'gamma': gamma_final,
        'model': lr,
        'metrics': {
            'accuracy': accuracy,
            'precision': precision,
            'recall': recall,
            'f1': f1,
            'auc': auc
        }
    }

def main():
    """
    Main execution function
    """
    print("IPCA EARNINGS SURPRISE PREDICTION")
    print("="*50)
    
    # Load and prepare data
    filepath = '/teamspace/studios/this_studio/goup_project_sample_v3.csv'  # Replace with your actual file path
    data, char_vars = prepare_earnings_surprise_data(filepath)
    
    # Select optimal K
    optimal_k, k_results = select_optimal_k(
        data, char_vars,
        k_range=range(4, 16),  # Test K from 4 to 15
        min_obs_per_date=100,
        oos_start_year=2012,
        oos_window=60
    )
    
    # Run IPCA analysis with optimal K
    print(f"\nRunning final analysis with K = {optimal_k}...")
    results, ipca, ipca_input = run_ipca_earnings_prediction(
        data, char_vars, 
        K=optimal_k,
        min_obs_per_date=100,
        oos_start_year=2012,
        oos_window=60
    )
    
    # Print IPCA results
    print(f"\nFINAL IPCA ESTIMATION RESULTS (K={optimal_k}):")
    print(f"Managed Portfolio R²: {results['xfits']['R2_Total']:.3f}")
    print(f"Returns R²: {results['rfits']['R2_Total']:.3f}")
    
    # Detailed evaluation with optimal K
    prediction_results = evaluate_ipca_factors_for_prediction(
        results, ipca_input, ipca, K=optimal_k, verbose=True
    )
    
    if prediction_results:
        # Save results
        prediction_results['factor_scores'].to_csv('ipca_factor_scores.csv', index=False)
        results['Gamma'].to_csv('ipca_factor_loadings.csv')
        
        # Save K selection results
        k_results_df = pd.DataFrame(k_results).T
        k_results_df.to_csv('k_selection_results.csv')
        
        print(f"\nResults saved:")
        print(f"- Factor scores: ipca_factor_scores.csv")
        print(f"- Factor loadings: ipca_factor_loadings.csv") 
        print(f"- K selection results: k_selection_results.csv")
        print(f"- Optimal K used: {optimal_k}")

if __name__ == '__main__':
    main()

IPCA EARNINGS SURPRISE PREDICTION
Loaded data: 911537 observations, 165 features
Earnings surprise distribution:
earnings_surprise
1.0    310507
NaN    307811
0.0    293219
Name: count, dtype: int64
Using 148 characteristic variables for IPCA

SELECTING OPTIMAL NUMBER OF FACTORS (K)

Testing K = 4...
Data after filtering: 403294 observations
Using 295 dates with sufficient cross-section
